--- Day 7: Some Assembly Required ---

This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates! Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.

Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from `0` to `65535`). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

The included instructions booklet describes how to connect the parts together: `x AND y -> z` means to connect wires `x` and `y` to an AND gate, and then connect its output to wire `z`.

For example:

- `123 -> x` means that the signal `123` is provided to wire `x`.
- `x AND y -> z` means that the bitwise AND of wire `x` and wire `y` is provided to wire `z`.
- `p LSHIFT 2 -> q` means that the value from wire `p` is left-shifted by `2` and then provided to wire `q`.
- `NOT e -> f` means that the bitwise complement of the value from wire `e` is provided to wire `f`.

Other possible gates include OR (bitwise OR) and RSHIFT (right-shift). If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.

For example, here is a simple circuit:

```
123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
```

After it is run, these are the signals on the wires:

```
d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
x: 123
y: 456
```

In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to wire `a`?


In [13]:
from __future__ import annotations
file_input = 'day7input.txt'

class Operation:
    def __init__(self, name: str, op_code: int, a: Operation|str, b: Operation|str):
        self.name = name
        self.op_code = op_code
        self.a = int(a) if a and a.isdecimal() else a
        self.b = int(b) if b and b.isdecimal() else b
        self.result = None

    # Resolve should be idempotent
    def resolve(self) -> int:
        a = None
        b = None
        #print(f'Evaluating {self.name}...')
        if self.result == None:
            #print('Not resolved, resolving children...')
            # Produce output by resolving children ( recursive )
            a = self.a.resolve() if type(self.a) == Operation else self.a
            b = self.b.resolve() if type(self.b) == Operation else self.b

            # Produce output based on results of inputs
            match(self.op_code):
                case 0:
                    # Immediate Value
                    self.result = a
                case 1:
                    # NOT
                    self.result = ~a
                case 2:
                    # OR
                    self.result = a | b
                case 3:
                    # AND
                    self.result = a & b
                case 4:
                    # LSHIFT
                    self.result = a << b
                case 5:
                    # RSHIFT
                    self.result = a >> b
        #print(f'{self.name} = {self.result}')
        return self.result

operations = {}
OP_STR = {
    'IMM'       : 0,
    'NOT '      : 1,
    ' OR '      : 2,
    ' AND '     : 3,
    ' LSHIFT '  : 4,
    ' RSHIFT '  : 5,
}
OP_KEYS = list(OP_STR.keys())

with open(file_input, 'r') as input:
    for line in input:
        # parse line
        op_code = -1
        in1 = None
        in2 = None
        op_txt, name = line.split(' -> ')
        name = name.strip()

        # Based on op code, get input values
        for op in OP_KEYS:
            if op in op_txt:
                op_code = OP_STR[op]
                inputs = op_txt.split(op)
                # Only populate in2 if there is a second input
                in1,in2 = inputs if op != 'NOT ' else (inputs[1], None)
        if op_code == -1:
            op_code = 0
            in1 = op_txt

        # Create operation object
        # If input is int, parse it as immediate value
        # If input is str, leave as is.
        # When all operations are parsed, connect numbers and references.
        ##print(f'Creating {OP_KEYS[op_code]} op called \'{name}\' with inputs {in1} and {in2}')
        operations[name] = Operation(name, op_code, in1, in2)

# Now that all operations are parsed, connect outputs to inputs

op:Operation
for op in list(operations.values()):
    if type(op.a) == str:
        # bind to operation
        other_op = operations.get(op.a, None)
        if other_op == None:
            raise KeyError(f'Variable \'{op.a}\' not found in file.')
        else:
            op.a = other_op
    if type(op.b) == str:
        # bind to operation
        other_op = operations.get(op.b, None)
        if other_op == None:
            raise KeyError(f'Variable \'{op.b}\' not found in file.')
        else:
            op.b = other_op

# All operations should be linked now. Lets try resolving the root operation, 'a'
print(f"Part 1: {operations['a'].resolve()}")

Part 1: 46065


# Part Two

Now, take the signal you got on wire a, override wire b to that signal, and reset the other wires (including wire a). What new signal is ultimately provided to wire a?

In [14]:
override = operations['a'].result

for op in list(operations.values()):
    op.result = None

operations['b'].result = override
print(f"Part 2: {operations['a'].resolve()}")

Part 2: 14134
